<a href="https://colab.research.google.com/github/Rajeeb321123/Machine-learning-Journey/blob/master/22_artificial_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Importing the libraries

In [162]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [163]:
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [164]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values # removing non important columns
y = dataset.iloc[:, -1].values

In [165]:
X[:2], y[:2]

(array([[619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
        [608, 'Spain', 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58]],
       dtype=object),
 array([1, 0]))

In [166]:
# Checking the missing values or NaN values
labels = dataset.columns.values.tolist()
missingValues=[]
for i, column in enumerate(labels[3:-1]):
    for j, value in enumerate(X[:,i]):
      if pd.isnull(value):
        print(f"At {j}th index of column {column} is NaN")
        missingValues.append([column, j])

print(f"The number of missing values are: {len(missingValues)}")

The number of missing values are: 0


### Encoding categorical data

Label Encoding the "Gender" column
> `We use Label encoder for Gender because only 2 values male or female`

In [167]:
X[:10]

array([[619, 'France', 'Female', 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 'Female', 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 'Female', 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [699, 'France', 'Female', 39, 1, 0.0, 2, 0, 0, 93826.63],
       [850, 'Spain', 'Female', 43, 2, 125510.82, 1, 1, 1, 79084.1],
       [645, 'Spain', 'Male', 44, 8, 113755.78, 2, 1, 0, 149756.71],
       [822, 'France', 'Male', 50, 7, 0.0, 2, 1, 1, 10062.8],
       [376, 'Germany', 'Female', 29, 4, 115046.74, 4, 1, 0, 119346.88],
       [501, 'France', 'Male', 44, 4, 142051.07, 2, 0, 1, 74940.5],
       [684, 'France', 'Male', 27, 2, 134603.88, 1, 1, 1, 71725.73]],
      dtype=object)

In [168]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = np.array(le.fit_transform(X[:,2])) # we need np array


In [169]:
X[:10]

array([[619, 'France', 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [608, 'Spain', 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [502, 'France', 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [699, 'France', 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [850, 'Spain', 0, 43, 2, 125510.82, 1, 1, 1, 79084.1],
       [645, 'Spain', 1, 44, 8, 113755.78, 2, 1, 0, 149756.71],
       [822, 'France', 1, 50, 7, 0.0, 2, 1, 1, 10062.8],
       [376, 'Germany', 0, 29, 4, 115046.74, 4, 1, 0, 119346.88],
       [501, 'France', 1, 44, 4, 142051.07, 2, 0, 1, 74940.5],
       [684, 'France', 1, 27, 2, 134603.88, 1, 1, 1, 71725.73]],
      dtype=object)

One Hot Encoding the "Geography" column

In [170]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])] , remainder='passthrough') # for 2nd column or Geography of X, index = 0 so [0]. remainder='passthrough' so unspecified column arenot dropped: alternative remainder='drop'

X = ct.fit_transform(X)
X = np.array(X) # ct.fi_transform donot return numpy array but later we are going to need numpy array

X[:10]

array([[1.0, 0.0, 0.0, 619, 0, 42, 2, 0.0, 1, 1, 1, 101348.88],
       [0.0, 0.0, 1.0, 608, 0, 41, 1, 83807.86, 1, 0, 1, 112542.58],
       [1.0, 0.0, 0.0, 502, 0, 42, 8, 159660.8, 3, 1, 0, 113931.57],
       [1.0, 0.0, 0.0, 699, 0, 39, 1, 0.0, 2, 0, 0, 93826.63],
       [0.0, 0.0, 1.0, 850, 0, 43, 2, 125510.82, 1, 1, 1, 79084.1],
       [0.0, 0.0, 1.0, 645, 1, 44, 8, 113755.78, 2, 1, 0, 149756.71],
       [1.0, 0.0, 0.0, 822, 1, 50, 7, 0.0, 2, 1, 1, 10062.8],
       [0.0, 1.0, 0.0, 376, 0, 29, 4, 115046.74, 4, 1, 0, 119346.88],
       [1.0, 0.0, 0.0, 501, 1, 44, 4, 142051.07, 2, 0, 1, 74940.5],
       [1.0, 0.0, 0.0, 684, 1, 27, 2, 134603.88, 1, 1, 1, 71725.73]],
      dtype=object)

### Splitting the dataset into the Training set and Test set

In [171]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1 )

In [172]:
X_train[:3], y_train[:3]

(array([[0.0, 1.0, 0.0, 628, 1, 29, 3, 113146.98, 2, 0, 1, 124749.08],
        [1.0, 0.0, 0.0, 626, 0, 29, 4, 105767.28, 2, 0, 0, 41104.82],
        [0.0, 1.0, 0.0, 612, 0, 47, 6, 130024.87, 1, 1, 1, 45750.21]],
       dtype=object),
 array([0, 0, 1]))

### Feature Scaling

`Feature Scaling is fundamentals all the vairables (everything) in ANN and Deep learning.`

In [173]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train) #Unlike Machine learning all variable is featured scaled in ANN and deep learning
X_test = sc.transform(X_test)

In [174]:
X_train[:3], y_train[:3]

(array([[-0.99850112,  1.71490137, -0.57273139, -0.23082038,  0.91509065,
         -0.94449979, -0.70174202,  0.58817274,  0.80225696, -1.55337352,
          0.97725852,  0.42739449],
        [ 1.00150113, -0.58312392, -0.57273139, -0.25150912, -1.09278791,
         -0.94449979, -0.35520275,  0.46984886,  0.80225696, -1.55337352,
         -1.02327069, -1.02548708],
        [-0.99850112,  1.71490137, -0.57273139, -0.3963303 , -1.09278791,
          0.77498705,  0.33787579,  0.85878772, -0.91150957,  0.64376017,
          0.97725852, -0.94479772]]),
 array([0, 0, 1]))

## Part 2 - Building the ANN

### Initializing the ANN

1. Sequence of layers (input to... output layers)
2. Object of Sequential class

In [175]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [176]:
# 6 = no of hidden neuron/units. Just based on experimentaion and trying several numbers
# Common activation function of fully connected layers and input layer is rectifier activation function
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))# add Dense: full connected layer which is object of dense class and automatically add input layer

### Adding the second hidden layer

In [177]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [178]:
# output layer need dimension of output layers
# if 3 outputs need 3 units/neuron
# for binary 0 or 1 , only neuron is ok
# Activation function for binary output layer is sigmoid and multiple output is softmax
# sigmoid: because return prediction and probability as well
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

Adam is a good stochastic gradient descent

In [179]:
ann.compile(optimizer = 'adam' ,
            loss = 'binary_crossentropy', # for binary classification. if non-binary = categorical_crossentropy
            metrics = ['accuracy'])

### Training the ANN on the Training set

In [180]:
ann.fit(X_train,
        y_train,
        batch_size = 32,# batch is more efficient, 32 in each batch
        epochs = 100
        )

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5955 - accuracy: 0.7040
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4635 - accuracy: 0.8039
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4434 - accuracy: 0.8051
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4356 - accuracy: 0.8075
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4299 - accuracy: 0.8077
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4235 - accuracy: 0.8121
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4172 - accuracy: 0.8165
Epoch 8/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4106 - accuracy: 0.8198
Epoch 9/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4046 - accuracy: 0.8214
Epoch 10/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3994 - accura

In [181]:
ann.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (32, 6)                   78        
                                                                 
 dense_8 (Dense)             (32, 6)                   42        
                                                                 
 dense_9 (Dense)             (32, 1)                   7         
                                                                 
Total params: 127 (508.00 Byte)
Trainable params: 127 (508.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

In [182]:
# create test
test_1 = np.array([[600, 'France', 'Male', 40, 3, 60000, 2, 1, 1,  50000]])
test_1

array([['600', 'France', 'Male', '40', '3', '60000', '2', '1', '1',
        '50000']], dtype='<U21')

In [183]:
# label encoding Gender
test_1[:,2] = np.array(le.transform(test_1[:,2])) # we need np array
test_1

array([['600', 'France', '1', '40', '3', '60000', '2', '1', '1', '50000']],
      dtype='<U21')

In [184]:
# Categorical encoding country
test_1 = ct.transform(test_1)
test_1

array([['1.0', '0.0', '0.0', '600', '1', '40', '3', '60000', '2', '1',
        '1', '50000']], dtype='<U32')

In [185]:
# Feature scaling
test_1 = sc.transform(test_1)
test_1

array([[ 1.00150113, -0.58312392, -0.57273139, -0.52046273,  0.91509065,
         0.10629772, -0.70174202, -0.26396987,  0.80225696,  0.64376017,
         0.97725852, -0.87097985]])

In [190]:
test_1_pred = ann.predict(test_1) # ouptut in probability because sigmoid activation function
test_1_pred

1/1 [==============================] - 0s 40ms/step


array([[0.04180796]], dtype=float32)

In [191]:
# in binary 0 or 1
test_1_pred = test_1_pred > 0.5
test_1_pred

array([[False]])

### Predicting the Test set results

In [213]:
y_pred = ann.predict(X_test) # ouptut in probability because sigmoid activation function

63/63 [==============================] - 0s 3ms/step


In [214]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [216]:
print(
    np.concatenate((
        y_pred.reshape(len(y_pred), 1),
        y_test.reshape(len(y_test), 1),
        ), axis = 1)
)

[[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [217]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1533   52]
 [ 228  187]]


0.86